In [10]:
import numpy as np
from tqdm import tqdm

### Define Distribution Functions

In [11]:
def N(mean, variance, size = None):
    return np.random.normal(mean, np.sqrt(variance), size)

def U(start_point, end_point, size = None):
    return np.random.uniform(start_point, end_point, size)

def B(α, β, size = None):
    return np.random.beta(α, β, size)

### Define Observation Functions

In [12]:
def transition_function(x̄, ϕ_x, σ_x, ξ_t, x):
    particles = x̄ + ϕ_x * (x - x̄) + σ_x * ξ_t
    return particles

In [13]:
def observation_likelihood(y, μ, x):
    σ_t = np.exp(x / 2)

    likelihood = (1 / (np.sqrt(2 * np.pi) * σ_t)) * np.exp(- ((y - μ)**2) / (2 * σ_t**2))

    return likelihood

In [14]:
y = [1,2,3,4,5]

In [15]:
L = 1_000 # number of particles
T = len(y) # number of time steps

In [16]:
μ = U(-5, 5, L) # shape (L, 1)
x̄ = U(-1, 5, L) # shape (L, 1)
σ_x = U(0, 2, L) # shape (L, 1)
ϕ_x = 2 * B(20, 1.5, L) - 1 # shape (L, 1)

In [ ]:
x = np.zeros((T + 1, L)) # shape (T + 1, L)
p = np.zeros((T, L)) # particles, shape (T, L)
α = np.zeros((T, L)) # shape (T, L)

### initialize $x_0$

In [18]:
x[0] = N(x̄, (σ_x ** 2) / (1 - ϕ_x ** 2))

### Particle Filtering Loop

In [ ]:
for t in tqdm(range(0, T, 1)):
    # generate system noise
    ξ_t = N(0, 1, L)
    # calculate particles from transition function
    p[t] = transition_function(x̄, ϕ_x, σ_x, ξ_t, x[t])
    # particles = x̄ + ϕ_x * (x[t-1] - x̄) + σ_x * ξ_t

    # calculate likelihood with Gaussian likelihood
    α[t] = observation_likelihood(y[t], μ, p[t])

    # calculate the weights as the normalized likelihoods
    weights = α[t]
    weights /= np.sum(α[t])

    # Resample latents from the particles
    for sample_j in range(L):
        sample_choice = np.random.choice(np.arange(L), p = weights)
        x[t + 1, sample_j] = p[t, sample_choice]


100%|██████████| 5/5 [00:00<00:00, 60.72it/s]
